# A2: Vector Semantics

Nikolai Ilinykh, Mehdi Ghanimifard, Wafia Adouane and Simon Dobnik

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read [the following instructions](https://github.com/sdobnik/computational-semantics/blob/master/README.md) on how to work on group assignments.

Write all your answers and the code in the appropriate boxes below.

In this lab we will look at how to build distributional semantic models from corpora and use semantic similarity captured by these models to do semantic tasks. We are also going to examine how different vector composition functions for phrases affect both the model and the learned information about similarities.  

Note that this lab uses a code from `dist_erk.py`, which contains functions that highly resemble those shown during the lecture. In the end, you can use either of the functions (from the lecture / from the file) to solve the tasks.

In [1]:
# the following command simply imports all the methods from that code.
from dist_erk import *

## 1. Loading a corpus

To train a distributional model, we first need a sufficiently large collection of texts which contain different words used frequently enough in different contexts. Here we will use a section of the Wikipedia corpus which you can download from [here](https://linux.dobnik.net/cloud/index.php/s/isMBj49jt5renYt?path=%2Fresources%2Fa2-distributional-representations) (wikipedia.txt.zip). (This file has been borrowed from another lab by [Richard Johansson](http://www.cse.chalmers.se/~richajo/)).  
When unpacked, the file is 151mb, hence if you are using the MLT servers you should store it in a temporary folder outside your home and adjust the `corpus_dir` path below.  
<!-- <It may already exist in `/opt/mlt/courses/cl2015/a5`.> -->


In [2]:
corpus_dir = './wikipedia/'

## 2. Building a model

Now you are ready to build the model.  
Using the methods from the code imported above build three word matrices with 1000 dimensions as follows:  

(i) with raw counts (saved to a variable `space_1k`);  
(ii) with PPMI (`ppmispace_1k`);  
(iii) with reduced dimensions SVD (`svdspace_1k`).  
For the latter use `svddim=5`. **[5 marks]**

Your task is to replace `...` with function calls. Functions are imported from `dist_erk.py` earlier, and they largely resemble functions shown during the lecture.

In [4]:
numdims = 1000
svddim = 5

# which words to use as targets and context words?
# we need to count the words and keep only the N most frequent ones
# which function would you use here with which variable?
ktw = do_word_count(corpus_dir,numdims)

wi = make_word_index(ktw) # word index
words_in_order = sorted(wi.keys(), key=lambda w:wi[w]) # sorted words

# create different spaces (the original matrix space, the ppmi space, the svd space)
# which functions with which arguments would you use here?
print('create count matrices')
space_1k = make_space(corpus_dir, wi, numdims)
print('ppmi transform')
ppmispace_1k = ppmi_transform(space_1k, wi)
print('svd transform')
svdspace_1k = svd_transform(ppmispace_1k, numdims, svddim)
print('done.')

reading file wikipedia.txt
create count matrices
reading file wikipedia.txt
ppmi transform
svd transform
done.


In [9]:
# now, to test the space, you can print vector representation for some words
print('house:', space_1k['house'])
print('life:', ppmispace_1k['life'])
print('nature:', svdspace_1k['nature'])

house: [2554 3774 3105  567  962  631  443  185  311  189  131   28   93  169
   81  125  151  408  194   90   79   29  217  184   62   15   31   70
   10    1   41   21    1   31   37    1   30    5   25    7    3   20
   11    1   32   36    2    5   66    4    0   46    8   18   28    0
   20    7    8   16   10   40    0  175   10    2    7   19    1  174
   11    3    1    6    0    0    0   10    9   11    7   24    4    4
   14   23   58    7    0   10    2    3   10    6   18    6   13    3
   22    0    3    5    3    7   14    3   40   20   19   15    6    8
   24    4    5    1   19    0    3    1    0   14    0   14   53    7
    7   11    6    5    5    4   12    6   53    1    1  433    4    0
    5    7    7   12    1    1    3    4   17    8   16    1    2   31
    1   12   14    1   44    6   14    9   38    7    2    6    8    1
   10    6   10    1    9    7    9    4    3   10    0   11    3    2
    0    2   11   37    2    0    2    1    5    9   10   16   88    6

Oxford Advanced Dictionary has 185,000 words, hence 1,000 words is not representative. We trained a model with 10,000 words, and 50 dimensions on truncated SVD. It took 40 minutes on a laptop. We saved all three matrices [here](https://linux.dobnik.net/cloud/index.php/s/isMBj49jt5renYt?path=%2Fresources%2Fa2-distributional-representations) (pretrained.zip). Download them and unpack them to a `pretrained` folder which should be a subfolder of the folder with this notebook:

In [10]:
import numpy as np

numdims = 10000
svddim = 50

print('Please wait...')
ktw_10k       = np.load('./pretrained/ktw_wikipediaktw.npy', allow_pickle=True)
space_10k     = np.load('./pretrained/raw_wikipediaktw.npy', allow_pickle=True).all()
ppmispace_10k = np.load('./pretrained/ppmi_wikipediaktw.npy', allow_pickle=True).all()
svdspace_10k  = np.load('./pretrained/svd50_wikipedia10k.npy', allow_pickle=True).all()
print('Done.')


Please wait...
Done.


In [11]:
# testing semantic space
print('house:', space_10k['house'])
print('life:', space_10k['life'])
print('historian:', space_10k['historian'])

house: [2554 3774 3105 ...    0    0    0]
life: [4502 2079 3365 ...    0    0    0]
historian: [693 183  77 ...   0   0   0]


## 3. Testing semantic similarity

The file `similarity_judgements.txt` (a copy is included with this notebook) contains 7,576 pairs of words and their lexical and visual similarities (based on the pictures) collected through crowd-sourcing using Mechanical Turk as described in [1]. The score range from 1 (highly dissimilar) to 5 (highly similar). Note: this is a different dataset from the phrase similarity dataset we discussed during the lecture (the one from [2]). For more information, please read the papers.

The following code will transform similarity scores into a Python-friendly format:

In [12]:
word_pairs = [] # test suit word pairs
semantic_similarity = [] 
visual_similarity = []
test_vocab = set()

for index, line in enumerate(open('similarity_judgements.txt')):
    data = line.strip().split('\t')
    if index > 0 and len(data) == 3:
        w1, w2 = tuple(data[0].split('#'))
        # it will check if both words from each pair exist in the word matrix.
        if w1 in ktw_10k and w2 in ktw_10k:
            word_pairs.append((w1, w2))
            test_vocab.update([w1, w2])
            semantic_similarity.append(float(data[1]))
            visual_similarity.append(float(data[2]))
        
print('number of available words to test:', len(test_vocab-(test_vocab-set(ktw))))
print('number of available word pairs to test:', len(word_pairs))
#list(zip(word_pairs, visual_similarity, semantic_similarity))

number of available words to test: 12
number of available word pairs to test: 774


Now we are going to test how the cosine similarity between vectors of each of the three spaces (normal space, ppmi, svd) compares with the human similarity judgements for the words in the similarity dataset. Which of the three spaces best approximates human judgements?

For comparison of several scores, we can use [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman's_rank_correlation_coefficient) which is implemented in `scipy.stats.spearmanr` [here](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.stats.spearmanr.html). The values of the Sperman correlation coefficient range from -1, 0 to 1, where 0 indicates no correlation, 1 perfect correaltion and -1 negative correlation. Hence, the greater the number the better the similarity scores align. The p values tells us if the coefficient is statistically significant. For this to be the case, it must be less than or equal to $< 0.05$.

Here is how you can calculate Pearson's correlation coefficient betweeen the scores of visual similarity and semantic similarity of the available words in the test suite:

In [13]:
from scipy import stats

rho, pval = stats.spearmanr(semantic_similarity, visual_similarity)
print("""Visual Similarity vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho, pval))


Visual Similarity vs. Semantic Similarity:
rho     = 0.7122
p-value = 0.0000


Let's now calculate the cosine similarity scores of all word pairs in an ordered list using all three matrices. **[6 marks]**

In [17]:
from scipy.spatial import distance

raw_similarities  = [1- distance.cosine(space_10k[w1], space_10k[w2]) for w1, w2 in word_pairs]
ppmi_similarities = [1- distance.cosine(ppmispace_10k[w1], ppmispace_10k[w2]) for w1, w2 in word_pairs]
svd_similarities  = [1- distance.cosine(svdspace_10k[w1], svdspace_10k[w2]) for w1, w2 in word_pairs]


Now, calculate correlation coefficients between lists of similarity scores and the real semantic similarity scores from the experiment. The scores of what model best correlates them? Is this expected? **[6 marks]**

In [18]:
rho1, pval1 = stats.spearmanr(semantic_similarity, raw_similarities)
print("""Raw similarities vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho1, pval1))

rho2, pval2 = stats.spearmanr(semantic_similarity, ppmi_similarities)
print("""Ppmi similarities vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho2, pval2))

rho3, pval3 = stats.spearmanr(semantic_similarity, svd_similarities)
print("""SVD similarities vs. Semantic Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho3, pval3))

Raw similarities vs. Semantic Similarity:
rho     = 0.1522
p-value = 0.0000
Ppmi similarities vs. Semantic Similarity:
rho     = 0.4547
p-value = 0.0000
SVD similarities vs. Semantic Similarity:
rho     = 0.4232
p-value = 0.0000


**Any comments/thoughts should go here:**

After calculating the correlation coffficients, we can see that the ppmi model correaltes the best, as it has the highest rho vaue, and the higher the rho value the greater the similarity. The p-value coefficient is statistically significant in all models since it's 0 (so less than 0.05).

We expected the ppmi model to present a better performance because ppmi takes into consideration the probability of an occurence. In other words, it penalises frequently occuring words across all contexts (similarly to tf-idf) and measures how distinctive the word is for a particular context. Ppmi will maintain information for those words that only add important information for the target word. 

We can also calculate correlation coefficients between lists of cosine similarity scores and the real visual similarity scores from the experiment. Which similarity model best correlates with them? How do the correlation coefficients compare with those from the previous comparison - and can you speculate why do we get such results? **[7 marks]**

In [19]:
rho4, pval4 = stats.spearmanr(visual_similarity, raw_similarities)
print("""Raw similarities vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho4, pval4))

rho5, pval5 = stats.spearmanr(visual_similarity, ppmi_similarities)
print("""Ppmi similarities vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho5, pval5))

rho6, pval6 = stats.spearmanr(visual_similarity, svd_similarities)
print("""SVD similarities vs. Visual Similarity:
rho     = {:.4f}
p-value = {:.4f}""".format(rho6, pval6))

Raw similarities vs. Visual Similarity:
rho     = 0.1212
p-value = 0.0007
Ppmi similarities vs. Visual Similarity:
rho     = 0.3838
p-value = 0.0000
SVD similarities vs. Visual Similarity:
rho     = 0.3097
p-value = 0.0000


**Any comments/thoughts should go here:**

Again the PPMI model works best, but all models are less efficient than before (meaning in similarity scores-real semantic similarity scores). P-value remains the same, apart from that in the Raw model, which means that it is less significant than the rest. That probably happens because semantic similarity provides more semantic information that derives from context (and a model can learn from context), whereas visual similarity is not based on context but rather on a visual representation of an object which can be more subjective from person to person. In terms of visual similarity, our models could be improved if trained using visual data, such as images.

## 4. Operations on similarities

We can perform mathematical operations on vectors to derive meaning predictions. For example, we can subtract the normalised vectors for `king` minus `queen` and add the resulting vector to `man` and we hope to get the vector for `woman`. Why? **[3 marks]**

We expect that "man" is closer in the vector space to "king" and "woman" is closer to "queen".
From a tehoretical point of view, we assume that the most accurate representation for that is King - Man + Woman = Queen. King and man share the same characteristic, which is [+masculine] and queen and woman share the [+feminine] characteristic. King and queen share the [+royal] characteristic. So according to our way of thinking, when we subtract the value man from the king, we subtract the [+masculine] characreristic and it becomes [-masculine]. After that we add the [+feminine] characteristic with the word woman and it turns from king to queen, as these two words share a same characteristic as well. 
What makes this question a bit trickier is the lack of parenthesis that control the order of operations. For instance, "King - Man + Woman = Queen" may be interpreted differently than "(King - Man) + Woman = Queen".
A final observation relies to the fact that if these correlations belong to different context, the interpretation will differ as well. If we study these terms diachornically (e.g., 1800 vs 2020), the semantic information varies from time to time. Thus, semantically these terms have a different weight.

Here is some helpful code that allows us to calculate such comparisons.

In [20]:
from scipy.spatial import distance

def normalize(vec):
    return vec / veclen(vec)

def find_similar_to(vec1, space):
    # vector similarity funciton
    #sim_fn = lambda a, b: 1-distance.euclidean(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.correlation(a, b)
    #sim_fn = lambda a, b: 1-distance.cityblock(normalize(a), normalize(b))
    #sim_fn = lambda a, b: 1-distance.chebyshev(normalize(a), normalize(b))
    #sim_fn = lambda a, b: np.dot(normalize(a), normalize(b))
    sim_fn = lambda a, b: 1-distance.cosine(a, b)

    sims = [
        (word2, sim_fn(vec1, space[word2]))
        for word2 in space.keys()
    ]
    return sorted(sims, key = lambda p:p[1], reverse=True)

Here is how you apply this code. Comment on the results you get. **[3 marks]**

In [21]:
short = normalize(svdspace_10k['short'])
light = normalize(svdspace_10k['light'])
long = normalize(svdspace_10k['long'])
heavy = normalize(svdspace_10k['heavy'])

find_similar_to(light - (heavy - long), svdspace_10k)[:10]

[('long', 0.8733111261346901),
 ('above', 0.8259671977311955),
 ('around', 0.8030776291120685),
 ('sun', 0.7692439111243973),
 ('just', 0.7678481974778111),
 ('wide', 0.767257431992253),
 ('each', 0.7665960260861158),
 ('circle', 0.7647746702909336),
 ('length', 0.7601066921319761),
 ('almost', 0.7542351860536628)]

**Any comments/thoughts should go here:**

Probably we expect something close to the "short" meaning, since we subtract "long". We have no clue on the existance of "sun" and "circle" semantic representation, however "sun" may have a semantic similarity with "light". "Length" and "wide" are probably corellated with "long". We also expected to see the term "tall", but maybe is not a frequent one (within the first 10 terms).


Find 5 similar pairs of pairs of words and test them. Hint: Google for `word analogies examples`. You can also construct analogies that are less lexical but more grammatical, e.g. `see, saw, leave, ?` or analogies that are based on world knowledge as in the [Google analogy dataset](http://download.tensorflow.org/data/questions-words.txt) from [3]. Does the resulting vector similarity confirm your expectations? But remember you can only do this if the words are contained in our vector space with 10,000 dimensions. **[10 marks]**

In [30]:
sun = normalize(svdspace_10k['sun'])
moon = normalize(svdspace_10k['moon'])
day = normalize(svdspace_10k['day'])
night = normalize(svdspace_10k['night'])
see = normalize(svdspace_10k['see'])
saw = normalize(svdspace_10k['saw'])
leave = normalize(svdspace_10k['leave'])
left = normalize(svdspace_10k['left'])
say = normalize(svdspace_10k['say'])
said = normalize(svdspace_10k['said'])

find_similar_to(sun - (moon - night), svdspace_10k)[:10] # 1
#find_similar_to(said - (say - left), svdspace_10k)[:10] # 2

[('night', 0.8886038339830423),
 ('morning', 0.8000957226947445),
 ('sun', 0.777799161109718),
 ('day', 0.7647056301917052),
 ('evening', 0.7575688171019718),
 ('watching', 0.75365192064434),
 ('sunday', 0.7464716284374743),
 ('end', 0.7453067996593553),
 ('room', 0.744111900775791),
 ('place', 0.7433668475829194)]

**Any comments/thoughts should go here:**
In example 1, the output presents a similar pattern as in the previous example. The third term used is "night", and the first output is "night" as well, and in the previous example the third term is "long" and the most frequent output is also "long". We expected the most frequent to be either "day" or "morning". These terms come in 2nd and 4th place. We cannot explain why "sun" is the 3rd more frequent word.

In example 2, we notice the same pattern as above, the first result that we get is that of the last word we give to the function, that is "left". More specifically, the pattern here is that all the results that we got are verbs of past tense. We believe that this is happening, because we substracted the word "say" from "said", which means we took off the neutral form of the verb and what is left is the past tense. The other verbs that we got as a result are quite similar semantically to each other.

## 5. Semantic composition and phrase similarity **[20 marks]**

In this task, we are going to look at how different semantic composition models, introduced in [2] correlate with human judgements. The file with the dataset is `mitchell_lapata_acl08.txt` included with this notebook.

Explanation of the task from Discord channel:

**What are we trying to achieve?**  
We want to create models, which can automatically capture differences in meaning of different phrases. These models should also be as good as we are (humans) in this task, right? Check example (1) from Mitchell/Lapata paper, it has two sentences which share the same words, but their meaning is completely different. We as humans can clearly see this difference, but how could a machine capture it?

It is intuitive that in order to get a meaning of a phrase, we might combine meaning of individual words in this phrase somehow, but how? First, we represent each word with the frequency vector based on the semantic space that we built before (e.g, frequency space, ppmi space, svd space). In other words, each word's meaning is represented by the number of times other words occur in the context, defined by window size. Now, we have a vector for `discussion` and a vector for `thrive`. How do we combine these vectors to get a single vector for the phrase `discussion thrive`?

Such methods of combining meaning vectors into a single item are called semantic composition methods (literally, because we compose semantic meaning of the phrase from its individuals). During the lecture, we tried different semantic composition methods: additive, multiplicative, combined.

Let's say we multiplied the vectors (went with the multiplicative method) and now we have one vector for our phrase.

Remember, we want to have a model that captures differences between the phrases; it means that if `discussion thrive` is our reference phrase, we need to get a different phrase to compare it with the reference one. How do we get this other phrase? Well, this other phrase can be either very similar to the reference or not similar at all, right? Let's say we decided to go with the second option and made/constructed a phrase `discussion digress`, which we know is very dissimilar to the reference phrase. We label this pair of phrases as having a low similarity, e.g., `low` in `hilo`. We can also create a different phrase (e.g., `discussion develop`) and use it as a high similarity phrase when paired with our reference phrase, right? This then would be labeled as `high` in `hilo`. This is what `hilo` in the dataset stands for: known information about how similar the reference phrase and the landmark phrase are.

Now, our main task is to automatically learn the similarities/differences between our reference and our landmark, right? We take the first pair: `discussion thrive` vs. `discussion digress`. We have a vector representation for each of these phrases. How do we compare them? We use cosine similarity to calculate a single score that would tell us about the similarity between these two vectors. The bigger the cosine, the more similar two vectors are. Cosine ranges from 0 to 1 (0 is very low similarity, 1 is very high similarity). Let's say, we get a cosine of 0.89, it means that our multiplicative model (remember, we decided to use multiplicative semantic composition method) thinks that these phrases are very similar (cosine is quite high). But wait a second, we know that these two phrases should be of low similarity, right? Because this is what the value in `hilo` tells us about this pair - `discussion thrive` and `discussion digress` are not similar to each other. Clearly, our multiplicative method fails to capture it. What can we do to improve our model? We can try a different composition method for getting a phrase vector from phrase's words: let's replace multiplication with addition. Or we can also use combined method. Let's say we used combined method and run cosine again; this time it tells that the cosine score is 0.45. Ok, this seems to be quite low, and it also agrees with our knowledge that these phrases are indeed not similar.

In other words, we need to evaluate different composition models (additive, multiplicative, combined, other methods from [2]) and analyse how well they perform. How do we analyse their performance? Because we have the ground-truth for comparison (`hilo` values), we know whether our phrases are actually similar or not. We want our cosine score to reflect this knowledge: if the score is high, but the groun-truth hilo is low, then we have a problem in the model - it did not learn things well, we need to replace the composition function.

_Long story short_: `hilo` is something that we compare our cosine to. `hilo` contains correct answers about similarity between phrases, and cosine should agree with this. If reference-landmark pair are `high` in `hilo`, then cosine should be high enough to reflecet that. If cosine is not high in this case, then we look at our model and change the composition function. We need to find the function, which give us cosines that are super close to the ground-truth known `hilo` values.

![title](./res.png)

Now, how do we compute something similar to the results from [2] (picture above):

In `High` and `Low` column you have mean cosine values.
These are calculated by basically averaging cosine scores for all pairs of phrases per model.
Rows introduce different models: `add` is additive, `multiply` is multiplicative, etc. (these are all described in the paper). `NonComp` is a baseline model, the most "stupid" one, it should be the worst. `UpperBound` is how humans performed in this task (they were asked to rate similarity between pairs of phrases, 1 is the lowest, 7 is the highest). Why these numbers are in a different scale, not from 0 to 1 like cosine, but from 1 to 7? Because they are not normalised, and authors explicitly said that they are interested in relative differences.

We need models which are closer to human ratings. `Add` model has 0.59 mean for `High` and 0.59 for `Low`, so it did not learn to differentiate between high similarity pairs and low similarity ones. This is a bad model then, we need a better one. `WeightAdd` seems to be doing better, the difference between `High` and `Low` is now 0.01, but it's also quite bad - the difference is not that obvious. The best models are `Multiply` and `Combined`, because their mean cosines for `High` and `Low` are quite different from each other. We can see that these two models gave higher cosines for high similarity pairs (0.42 and 0.38), while giving lower cosines for low similarity pairs (0.28 and 0.28). And this is a good result - it shows that these two composition functions are so far the best in (i) giving high cosine go highly similar pairs and low cosine to very dissimilar pairs, and (ii) keeping the distance (range) between high and low cosines quite large. However, we can't say how far/close they are when compared to human performance (UpperBound) since human scores are not normalised.

Still, we want to choose a model which is the closest to humans. This is why we are using the correlation test.

How do we perform the correlation test? We need to see how well *each model* correlated with human judgements. So for each model, we would have a vector of cosine values this models gives for each pair that we have. For example, let's say we have three pairs of phrases and our cosine values from additive model are the following ones: `[0.89, 0.40, 0.70]`. Now we need to get a vector of the same size, but for human scores. What do we have for human scores? We have multiple participants, which means that a single phrase can be evaluated by multiple participants. Let's say, the first phrase has scores from two participants (`6, 7`), so what we would do is that we would average it to have a single number (`(6 + 7) / 2 = 6.5`). With this, we can get a mean vector of human scores per item: `[6.5, 3, 6]`.

Now, we have two vectors and we can runk Spearman correlation on these vectors. This is what exactly what the third column in the Results table is showing (the correlation value). There is also a p-value (denoted right below the table).

**What is your ultimate task in this part of the assignment?**

(i) process the dataset and extract `reference - landmark` pairs; you can use the code from the lecture as something to start with. Try to keep information about human rating (`input`) and high/low similarity (`hilo`), because you will need it for correlation tests. Also, you might want to keep the information about participant id (will be useful for getting average numbers for correlation tests). Which format you should use to keep all this data? It's up to you, but a dictionary-like format could be a good choice.

(ii) build models of semantic phrase composition: in the lecture we introduced simple additive, simple multiplicative and combined models (details are in [2]). Your task is to take a single pair of phrases, and compute the composition of its vectors using each of these functions. Thus, you will have (at least) three compositional models that take each `noun - verb` phrase from the pair (these phrases can be either references or landmarks) and output a single vector, representing the meaning of this phrase. As your semantic space, you can use pretrained spaces (standard space, ppmi or svd) introduced above. It is up to you which space you use, but for someone who runs your code, it should be pretty straightforward to switch between them.

(iii) calculate Spearman correlation between each model's predictions and human judgements; you should have something similar to the scores that are shown in the paper.

**Thoughts process behind calculating the correlation:**

Let's look at the example pair: reference `child chatter` and high-level similarity landmark (as the last word in the row indicates) `child gabble`. Let's say we have 3 humans evaluating the similarity between these two phrases and we combine their scores into a single vector: `[5, 6, 5]`. We need to average them to get our human vector for correlation: `[5.3]`.

Our A model's output:  
`cosine(p1, p2) = 0.88`, where p1 is the result of addition of word vectors in the reference phrase `child gabble`, and p2 is the result of addition of word vectors in the high-level similarity phrase `child chatter`.  

Therefore, we have human rating vector `[5.3]` and model A output `[0.88]`. Next is to compute correlation between these two vectors. This should give you a correlation value and p-value for the model of choice and human ratings.

Of course, your human rating vectors will be longer (e.g., [6, 7, 3, 4, 5]). Each of your models (A, B, C) will produce a single vector of cosine similarity between these same pairs (e.g., [0.89, 0.98, 0.23, 0.65, 0.55]). The goal is to compare each model's cosine similarity vectors with human rating vectors and identify the model which outputs the best result in terms of being the closest to the way human rate similarity between the phrases.

The minimum to do in this task: compute correlations for at least ONE model and human ratings. However, this should not be hard to run it for any other model as well. For examples on how to interpret the results, look at Section 5 Results of the original paper.

In [42]:
# i) process dataset

with open('./mitchell_lapata_acl08.txt', 'r') as file:
    dataset = file.readlines()

for_model = []
human_judg = []

def judgements():
    
    for line in dataset[1:]:
        x = line.strip().split(' ')
        participant = x[0]
        ref1 = x[2],x[1]
        ref2 = x[2],x[3]
        in_put = x[4]
        hilo = x[5]
        if ref1[0] in ktw_10k and ref1[1] in ktw_10k and ref2[1] in ktw_10k: #if noun - verb do not exist in the words used to create ppmi model, it will raise an error in next function
            for_model.append([participant, tuple(ref1), tuple(ref2), in_put, hilo])
                
                
    for element in for_model: #find the human average for every phrase
        human = 0
        index = 0
        for line in dataset[1:]:
            if element[1][0] in line and element[1][1] in line and element[2][1] in line :
                x = line.strip().split(' ')
                human += int(x[4])
                index += 1
        human_judg.append(human/index)


judgements()




# ii) create Additive, Multiplicative, Combined models
    

add_cos = []
mult_cos = []
comb_cos = []



def additive(noun_vector,verb_vector,landmark_vector):
    add1_pair = noun_vector + verb_vector
    add2_pair = noun_vector + landmark_vector
    add_cos.append(1- distance.cosine(add1_pair, add2_pair)) #cosine value

def multiplicative(noun_vector,verb_vector,landmark_vector):
    mult1_pair = noun_vector * verb_vector
    mult2_pair = noun_vector * landmark_vector
    mult_cos.append(1- distance.cosine(mult1_pair, mult2_pair)) #cosine value

def combined(noun_vector, verb_vector, landmark_vector, alpha, beta, theta):
    comb1_add_pair = (alpha * noun_vector) + (beta * verb_vector)
    comb2_add_pair = (alpha * noun_vector) + (beta * landmark_vector)
    comb1_mult_pair = theta * noun_vector * verb_vector
    comb2_mult_pair = theta * noun_vector * landmark_vector
    p_reference = comb1_add_pair + comb1_mult_pair
    p_landmark = comb2_add_pair + comb2_mult_pair
    comb_cos.append(1- distance.cosine(p_reference, p_landmark)) #cosine value

    
# We used the ppmi space because it presented the best results in the previous task

space = ppmispace_10k # can be changed to other model as well
 
# In order to find how similar 2 phrases are, we need to find the vectors of their components
# and then combine those vectors

for element in for_model: #feed the model with specific words retrieved from the dataset list
    noun_vector = space[element[1][0]] # find vector for the noun
    verb_vector = space[element[1][1]] # find vector for the verb1
    landmark_vector = space[element[2][1]] # find vector for the verb2 
    additive(noun_vector,verb_vector,landmark_vector) # add model
    multiplicative(noun_vector,verb_vector,landmark_vector) # mult model
    combined(noun_vector, verb_vector, landmark_vector, alpha=0, beta=0.7, theta=0.3) # comb model
    
    

#iii) find models' correlations with human judgement

rho_add, pval_add = stats.spearmanr(add_cos, human_judg)
print("""Additive model vs. Human judgement:
rho     = {:.4f}
p-value = {:.4f}""".format(rho_add, pval_add))

rho_mult, pval_mult = stats.spearmanr(mult_cos, human_judg)
print("""Multiplicative model vs. Human judgement:
rho     = {:.4f}
p-value = {:.4f}""".format(rho_mult, pval_mult))

rho_comb, pval_comb = stats.spearmanr(comb_cos, human_judg)
print("""Combined model vs. Human judgement:
rho     = {:.4f}
p-value = {:.4f}""".format(rho_comb, pval_comb))


Additive model vs. Human judgement:
rho     = 0.6565
p-value = 0.0000
Multiplicative model vs. Human judgement:
rho     = 0.5691
p-value = 0.0000
Combined model vs. Human judgement:
rho     = 0.6759
p-value = 0.0000


**Any comments/thoughts should go here:**
1) In the "mitchell_lapata_acl08.txt", there are phrases in which the "hilo" does not correspond to the input rate (e.g., participant20 erupt fountain burst 1 high). If this is a mistake, it may affect the final average when calculating human judgement.
2) In paper [2], they concluded that the most efficient weighting constants for the chosen parameters are 0.95 for beta, 0.05 for theta and 0.0 for alpha. However, when we used these weights in our experiment, the rho value for the combined model was 0.3813, making this model the worst of all. It seems that when beta is set to 0.7 and theta to 0.3, the best results are achieve (see above), making the combined our best model for semantic correlation. 

# Literature

  - [1] C. Silberer and M. Lapata. Learning grounded meaning representations with autoencoders. In Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics, pages 721–732, Baltimore, Maryland, USA, June 23–25 2014 2014. Association for Computational Linguistics.  

  - [2] Mitchell, J., & Lapata, M. (2008). Vector-based Models of Semantic Composition. In Proceedings of ACL-08: HLT (pp. 236–244). Association for Computational Linguistics.
  
  - [3] T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.

## Marks

This assignment has a total of 60 marks.